In [1]:
import pandas as pd
import numpy as np

In [7]:
data_train = pd.read_csv('Train_modified.csv',parse_dates=['DATOP','STA','STD'])
data_train.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,AC,target,ATA,...,month_DATOP,year_DATOP,weekdays_DATOP,flight_duration_hrs,flight_duration_category,fl_op,ac_type,flight_length_cat,flight_duration_bin,time_of_day_cat
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12:55:00,TU 32AIMN,260.0,True,...,1,2016,6,2.416667,long,1.0,1.0,medium,120-179,morning
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16:55:00,TU 31BIMO,20.0,True,...,1,2016,2,1.833333,medium,1.0,2.0,medium,60-119,afternoon
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,TU 32AIMN,0.0,True,...,1,2016,5,2.583333,long,1.0,1.0,medium,120-179,night
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,TU 736IOK,0.0,True,...,1,2016,6,2.833333,long,1.0,3.0,medium,120-179,midday
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15:50:00,TU 320IMU,22.0,True,...,1,2016,6,1.333333,medium,1.0,1.0,short,60-119,midday


In [8]:
df = data_train.copy()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107833 entries, 0 to 107832
Data columns (total 21 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   ID                        107833 non-null  object        
 1   DATOP                     107833 non-null  datetime64[ns]
 2   FLTID                     107833 non-null  object        
 3   DEPSTN                    107833 non-null  object        
 4   ARRSTN                    107833 non-null  object        
 5   STD                       107833 non-null  datetime64[ns]
 6   STA                       107833 non-null  datetime64[ns]
 7   AC                        107833 non-null  object        
 8   target                    107833 non-null  float64       
 9   ATA                       107833 non-null  bool          
 10  SCH                       107833 non-null  bool          
 11  month_DATOP               107833 non-null  int64         
 12  ye